<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/scripts/Finetuning(Unsloth).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Finetuning with Unsloth

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-loiv634r/unsloth_d6b3bf666be1454b9f5d10bb84762f7c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-loiv634r/unsloth_d6b3bf666be1454b9f5d10bb84762f7c
  Resolved https://github.com/unslothai/unsloth.git to commit 56392a7a4c1bf38154a144854a61b7d2d4dd2d0b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.5/511.5 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 133.8 MB/s eta 0:00:00

In [ ]:
import torch
from datasets import load_dataset, DatasetDict
from huggingface_hub import login


In [ ]:
login()

In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import AutoProcessor
from peft import LoraConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving validation.csv to validation.csv


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch

# Load your CSVs
data_files = {
    "train": "train.csv",
    "validation": "validation.csv",
    "test": "test.csv",
}
dataset = load_dataset("csv", data_files=data_files)

# Inspect
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
})


In [ ]:

dataset

DatasetDict({
    train: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['disease', 'text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
# 🧴 Extract unique label names safely
label_values = sorted(list(set(dataset['train']['label'])))
formatted_classes = [f"{chr(65 + i)}: {label}" for i, label in enumerate(label_values)]

# Create the prompt for text classification
options = "\n".join(formatted_classes)
PROMPT = f"What is the most likely skin disease described in the text?\n{options}"

# 💬 Format function for text classification
def format_data(example: dict) -> dict:
    label_value = example["label"]
    example["messages"] = [
        {
            "role": "user",
            "content": [{"type": "text", "text": f"{PROMPT}\n\nDescription: {example['text']}"}],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": str(label_value)}],
        },
    ]
    return example

# 🧠 Apply formatting to all splits
formatted_data = dataset.map(format_data, remove_columns=["label"])

print("\n✅ Dataset is formatted for training.")
formatted_data


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


✅ Dataset is formatted for training.


DatasetDict({
    train: Dataset({
        features: ['disease', 'text', 'messages'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['disease', 'text', 'messages'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['disease', 'text', 'messages'],
        num_rows: 2000
    })
})

In [ ]:
model_id = "google/medgemma-4b-it"
model, processor = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=2048,
    dtype=None,             # Unsloth handles dtype automatically
    load_in_4bit=True,      # This enables 4-bit quantization
)
processor.tokenizer.padding_side = "right"
print("Model loaded with Unsloth optimizations.")

==((====))==  Unsloth 2025.11.1: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Model loaded with Unsloth optimizations.


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear",
    use_gradient_checkpointing=True,
)

def collate_fn(examples: list[dict]):
    images = [example['image'] for example in examples]
    texts = [
        processor.apply_chat_template(ex['messages'], add_generation_prompt=False, tokenize=False).strip()
        for ex in examples
    ]
    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
    labels = batch['input_ids'].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    batch["labels"] = labels
    return batch

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [11]:
from trl import SFTTrainer, SFTConfig
import torch


In [12]:
# 1️⃣ Define the formatting function to prepare a 'text' column
# This converts the 'messages' field into a single prompt-response text
def format_dataset_for_unsloth(example):
    """
    Takes a single data sample and creates a 'text' field with the
    fully formatted prompt string, using the processor.
    """
    # Convert structured messages into plain text
    text = processor.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}


# 2️⃣ Apply the formatting to your entire dataset
# This adds a new 'text' column used by the trainer
preformatted_dataset = formatted_data.map(
    format_dataset_for_unsloth,
    num_proc=1  # safer for Colab
)

print("✅ Dataset has been pre-formatted with a 'text' column.")
print("Example text entry:\n", preformatted_dataset["train"][0]["text"])


# 3️⃣ Define training arguments
args = SFTConfig(
    output_dir="medgemma-unsloth",
    num_train_epochs=7,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    save_strategy="epoch",
    eval_strategy='steps',
    seed=42,
    report_to="none",  # disable wandb for Colab
)


# 4️⃣ Prepare train and validation splits
train_data = preformatted_dataset["train"]
val_data = preformatted_dataset["validation"]


# 5️⃣ Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    dataset_text_field="text",  # specify your pre-formatted column
    max_seq_length=2048,
    dataset_num_proc=1,  # single-threaded for Colab stability
    args=args,
)


Map (num_proc=1):   0%|          | 0/3000 [00:00<?, ? examples/s]

Map (num_proc=1):   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=1):   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Dataset has been pre-formatted with a 'text' column.
Example text entry:
 <bos><start_of_turn>user
What is the most likely skin disease described in the text?
A: 0
B: 1

Description: Blistering or weeping lesions in acute flares appearing in skin creases and folds. The patient feels unbearably itchy, especially at night and describes the disease as mild and intermittent. Sometimes flares after infections or insect bites.<end_of_turn>
<start_of_turn>model
1<end_of_turn>

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/3000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
import random
from IPython.display import display, Markdown

# Pick a split (e.g., train)
split = dataset["train"]

# View, say, 9 random samples
num_samples = 9
examples = random.sample(range(len(split)), num_samples)

for idx in examples:
    example = split[idx]
    text = example["text"]
    label = example["label"]

    display(Markdown(f"### 🧾 Sample {idx}\n**Label:** `{label}`\n**Description:** {text}\n---"))


### 🧾 Sample 2619
**Label:** `1`
**Description:** The patient has intensely itchy, dry and scaly patches of skin located on the palms and soles; the person reports constant discomfort and burning sensation. The condition is severe and persistent and often worse in dry or cold weather.
---

### 🧾 Sample 456
**Label:** `1`
**Description:** Symptoms include thickened, leathery skin from repeated scratching at on the hands and fingers. The individual experiences social discomfort and reduced quality of life — the eczema is chronic, requiring ongoing moisturization and care and often worse in dry or cold weather.
---

### 🧾 Sample 102
**Label:** `1`
**Description:** Blistering or weeping lesions in acute flares appearing in skin creases and folds. The patient feels unbearably itchy, especially at night and describes the disease as variable, with periods of calm and sudden flare-ups. Often flares after contact with irritants or chemicals.
---

### 🧾 Sample 1126
**Label:** `1`
**Description:** On the trunk and limbs show red, inflamed rash with small bumps; patient notes they experiences sharp stinging when skin is broken. This is typically moderate with frequent flares and can be aggravated by certain fabrics like wool.
---

### 🧾 Sample 1003
**Label:** `1`
**Description:** Symptoms include blistering or weeping lesions in acute flares at on the wrists and ankles. The individual feels restless and loses sleep due to the itch — the eczema is chronic, requiring ongoing moisturization and care and often flares after contact with irritants or chemicals.
---

### 🧾 Sample 914
**Label:** `1`
**Description:** Symptoms include areas of skin that are tender and sore at on the hands and fingers. The individual feels unbearably itchy, especially at night — the eczema is mild and intermittent and may be triggered by hot showers or harsh cleansers.
---

### 🧾 Sample 571
**Label:** `1`
**Description:** Presentation: patches of skin that are rough and scaly on around the eyes and eyelids. Patient has pain and tenderness from cracks and fissures. Course: moderate with frequent flares; commonly can worsen during periods of hormonal change.
---

### 🧾 Sample 419
**Label:** `1`
**Description:** Inside the elbows and behind the knees show thickened, leathery skin from repeated scratching; patient notes they has pain and tenderness from cracks and fissures. This is typically moderate with frequent flares and sometimes flares after infections or insect bites.
---

### 🧾 Sample 2771
**Label:** `1`
**Description:** Symptoms include skin that flakes, crusts, or forms scabs at inside the elbows and behind the knees. The individual experiences sharp stinging when skin is broken — the eczema is mild and intermittent and linked to exposure to allergens like dust and pollen.
---

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 7 | Total steps = 2,625
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 38,497,792 of 4,338,577,264 (0.89% trained)


Step,Training Loss,Validation Loss
1,1.293000,5.117789
2,1.265600,4.963217
3,1.222700,4.631912
4,1.147600,4.311161
5,1.054700,4.006146
6,1.003900,3.689110


Unsloth: Not an error, but Gemma3ForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
